#  Pandas 데이터 로딩, 저장, 파일 형식

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('pydata/ch06/ex1.csv')
data

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv('pydata/ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv('pydata/ch06/ex2.csv',
           names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [484]:
pd.read_csv('pydata/ch06/ex2.csv', 
           names=['a','b','c','d','message'],
           index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,NaN,7,8
foo,9,10,11,12


계층 색인

In [486]:
pd.read_csv('pydata/ch06/csv_mindex.csv',
           index_col=['key1','key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

구분자가 잘 설정되지 않은 파일 읽기

In [4]:
pd.read_csv('pydata/ch06/ex3.txt', sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


특정 열 제외하고 읽어오기

In [6]:
def get_skip_rows(filename, pattern):
    """지정한 문자로 시작하는 행 번호 탐지
    Parameters
        filename : 파일명
        pattern: 탐지 대상 문자열.
    
    Returns: 해당하는 행 번호 (0-기반)
    """
    # 파일 내용 읽어오기
    with open(filename) as f:
        lines = f.readlines()
    # 패턴으로 지정된 행 번호 탐지
    skiprows = [i for i, l in enumerate(lines) 
                if l.startswith(pattern)]
    return skiprows

get_skip_rows('pydata/ch06/ex4.csv', '#')

[0, 2, 3]

In [7]:
filename = 'pydata/ch06/ex4.csv'
# skiprows = get_skip_rows(filename, '#')
pd.read_csv(filename, skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv?

누락된 데이터나 이상한 문자

In [503]:
df = pd.read_csv('pydata/ch06/ex5.csv')
df

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [504]:
df.isnull()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [505]:
df = pd.read_csv('pydata/ch06/ex5.csv', na_values=['NULL'])
df

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [508]:
pd.read_csv('pydata/ch06/ex5.csv',
              na_values={'message': ['foo', 'NA'],
                        'something': ['two']})

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11,12,NaN


## 텍스트 파일 일부 읽기

In [509]:
pd.read_csv('pydata/ch06/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


크기가 큰 파일은 일정한 크기로 나눈 덩어리(chunk)로 읽어들일 수 있다.

In [2]:
import pandas as pd
pd.read_csv?

In [512]:
chunks = pd.read_csv('pydata/ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunks:
    tot= tot.add(piece['key'].value_counts(), fill_value=0)

tot.order(ascending=False)[:10]

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
dtype: float64

## 데이터 텍스트 출력

In [2]:
data = pd.read_csv('pydata/ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [5]:
data.to_csv('out.csv', sep='|')

파일시스템 명령 수행

In [6]:
!cat out.csv

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


## JSON

In [19]:
obj = """
{
    "name":"Wes",
    "places_lived": ["USA", "Spain", "Germany"],
    "pet": null,
    "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
            {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""
obj

'\n{\n    "name":"Wes",\n    "places_lived": ["USA", "Spain", "Germany"],\n    "pet": null,\n    "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},\n            {"name": "Katie", "age": 33, "pet": "Cisco"}]\n}\n'

In [20]:
import json
result = json.loads(obj)
result

{u'name': u'Wes',
 u'pet': None,
 u'places_lived': [u'USA', u'Spain', u'Germany'],
 u'siblings': [{u'age': 25, u'name': u'Scott', u'pet': u'Zuko'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

In [21]:
type(result)

dict

In [23]:
json.dumps(result)

'{"pet": null, "siblings": [{"pet": "Zuko", "age": 25, "name": "Scott"}, {"pet": "Cisco", "age": 33, "name": "Katie"}], "name": "Wes", "places_lived": ["USA", "Spain", "Germany"]}'

In [25]:
DataFrame(result['siblings'], columns=['name', 'age'])

,name,age
0,Scott,25
1,Katie,33


## 웹페이지에서 정보 추출

In [42]:
from urllib2 import urlopen

res = urlopen('http://finance.yahoo.com/q/op?s=APPL+Options')

In [31]:
for line in res:
    print(line)

<!DOCTYPE html>

<html>

<head>

    <!-- customizable : anything you expected.  -->

    <title>APPL Option Chain | Yahoo! Inc. Stock - Yahoo! Finance</title>



    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />

    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />







    

        <link rel="stylesheet" type="text/css" href="http://l.yimg.com/zz/combo?/os/mit/td/stencil-0.1.306/stencil-css/stencil-css-min.css&/os/mit/td/finance-td-app-mobile-web-2.0.402/css.master/css.master-min.css"/><link rel="stylesheet" type="text/css" href="https://s.yimg.com/os/mit/media/m/quotes/quotes-search-gs-smartphone-min-1680382.css"/><link rel="stylesheet" type="text/css" href="https://s.yimg.com/os/stencil/3.1.0/styles-ltr.css"/><link rel="stylesheet" type="text/css" href="https://s.yimg.com/os/finance/dd-icon/1.0.19/yahoo-finance-icons.css"/>





<script>(function(html){var c = html.className;c += " JsEnabled";c = c.replace("NoJs","");html.className = c;})(

In [33]:
import requests

res = requests.get('http://finance.yahoo.com/q/op?s=APPL+Options')
print(res.text)

<!DOCTYPE html>
<html>
<head>
    <!-- customizable : anything you expected.  -->
    <title>APPL Option Chain | Yahoo! Inc. Stock - Yahoo! Finance</title>

    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />



    
        <link rel="stylesheet" type="text/css" href="http://l.yimg.com/zz/combo?/os/mit/td/stencil-0.1.306/stencil-css/stencil-css-min.css&/os/mit/td/finance-td-app-mobile-web-2.0.402/css.master/css.master-min.css"/><link rel="stylesheet" type="text/css" href="https://s.yimg.com/os/mit/media/m/quotes/quotes-search-gs-smartphone-min-1680382.css"/><link rel="stylesheet" type="text/css" href="https://s.yimg.com/os/stencil/3.1.0/styles-ltr.css"/><link rel="stylesheet" type="text/css" href="https://s.yimg.com/os/finance/dd-icon/1.0.19/yahoo-finance-icons.css"/>


<script>(function(html){var c = html.className;c += " JsEnabled";c = c.replace("NoJs","");html.className = c;})(document.docume

In [43]:
from lxml.html import parse

parsed = parse(res)
doc = parsed.getroot()
doc

<Element html at 0x106d54e10>

In [45]:
links = doc.findall('.//a')
links[:10]

[<Element a at 0x106d5d050>,
 <Element a at 0x106d5d0a8>,
 <Element a at 0x106d5d100>,
 <Element a at 0x106d5d158>,
 <Element a at 0x106d5d1b0>,
 <Element a at 0x106d5d208>,
 <Element a at 0x106d5d260>,
 <Element a at 0x106d5d2b8>,
 <Element a at 0x106d5d310>,
 <Element a at 0x106d5d368>]

In [48]:
links[0].get('href')

'https://www.yahoo.com/'

In [49]:
links[0].text_content()

'Home'

In [51]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[:10]

['https://www.yahoo.com/',
 'https://mail.yahoo.com/?.intl=us&.lang=en-US&.src=ym',
 'https://search.yahoo.com/search',
 'http://news.yahoo.com/',
 'http://sports.yahoo.com/',
 'http://finance.yahoo.com/',
 'https://weather.yahoo.com/',
 'https://games.yahoo.com/',
 'https://answers.yahoo.com/',
 'https://screen.yahoo.com/']

#### 도전과제

주식 종목 5개 코드베이직, SJ전자, 매크로소프트, 메론, 성주모직의 90일간의 가격 정보에 대해 다음을 수행한다. 각 주식의 가격은 천원 단위로 기록되어 있다.

a. 주식 종목 가격 정보를 JSON 문자열로 변환하시오.

b. JSON 문자열로된 주식 종목 가격을 텍스트 파일 stocks_90_days.txt에 저장하시오.

c. b의 텍스트 파일에서 읽어들인 정보를 파이썬의 dict으로 생성.

d. b의 텍스트 파일에서 읽어들인 정보를 pandas.DataFrame으로 생성.

In [19]:
import numpy as np
from pandas import DataFrame, Series

def gen_sim_price(num_stocks=10, num_days=90):
    """주식 종목 가격 시뮬레이션. 
    지정된 종목 개수에 대한 지정된 일수만큼의 모의 가격 생성"""
    prices = np.random.randint(5,1000, size=num_stocks)
    changes = 1+np.random.uniform(low=-0.3, high=0.3, 
                                  size=(num_days, num_stocks))
    prices =  prices * changes.cumprod(0)

    return prices

prices = gen_sim_price(5) # 5개 종목 90일 가격 생성
stock_names = [u'코드베이직', u'SJ전자', u'매크로소프트', u'메론', u'성주모직']
price_frame = DataFrame(prices,
                       columns=stock_names)
price_frame

,코드베이직,SJ전자,매크로소프트,메론,성주모직
0,509.703032,572.001032,336.002310,183.827369,114.606302
1,386.143945,448.835777,379.062771,137.068004,90.365824
2,313.123138,372.673549,366.544301,121.089417,81.349908
3,254.322493,323.809041,385.376949,111.518313,99.195890
4,230.425589,388.177909,461.009464,119.869294,124.181059
5,178.124247,388.363285,332.679160,89.353564,120.870733
6,216.370443,427.957503,288.422726,67.676954,128.232200
7,240.807515,541.880611,231.887499,47.433739,107.016389
8,203.092245,387.846094,259.884753,51.070995,104.554324
9,225.853868,385.940469,312.118196,44.043782,132.043436


In [24]:
# DataFrame을 JSON 문자열로 변환하기
price_json_str = price_frame.to_json()
# 파일에 문자열 저장
with open('stocks_90days.txt', 'w') as f:
    f.write(price_json_str.encode('utf-8'))

In [28]:
import json
# 파일에서 JSON 문자열 읽기
with open('stocks_90days.txt') as f:
    json_str = f.read()
# JSON 문자열을 dict으로 변환
json_dict = json.loads(json_str) 
json_dict

{u'SJ\uc804\uc790': {u'0': 572.0010323761,
  u'1': 448.8357772478,
  u'10': 448.6127684063,
  u'11': 542.3053567347,
  u'12': 529.5223780316,
  u'13': 534.3582223396,
  u'14': 678.2797967414,
  u'15': 567.9330671723,
  u'16': 450.4809700998,
  u'17': 397.9422817896,
  u'18': 435.0266343322,
  u'19': 312.0686931225,
  u'2': 372.6735492432,
  u'20': 353.5804284724,
  u'21': 269.8392584376,
  u'22': 241.6624158367,
  u'23': 184.1146600781,
  u'24': 178.4141393781,
  u'25': 177.1127425361,
  u'26': 202.5929867488,
  u'27': 211.7525583771,
  u'28': 251.6108600576,
  u'29': 315.5229319857,
  u'3': 323.8090406902,
  u'30': 405.8740428627,
  u'31': 487.5345951925,
  u'32': 397.875348268,
  u'33': 298.2530336729,
  u'34': 383.0704155136,
  u'35': 365.8919712131,
  u'36': 423.5208124921,
  u'37': 466.6218807429,
  u'38': 599.2241741038,
  u'39': 534.3282493159,
  u'4': 388.1779093245,
  u'40': 658.9226326166,
  u'41': 499.4812450292,
  u'42': 517.1346026977,
  u'43': 594.0106302134,
  u'44': 514

In [25]:
frame = pd.read_json('stocks_90days.txt')
frame

,SJ전자,매크로소프트,메론,성주모직,코드베이직
0,572.001032,336.002310,183.827369,114.606302,509.703032
1,448.835777,379.062771,137.068004,90.365824,386.143945
10,448.612768,334.007863,35.299311,122.042719,292.024167
11,542.305357,321.064986,27.240605,124.252719,340.267098
12,529.522378,318.509388,20.436602,98.041802,374.297634
13,534.358222,353.060370,22.664216,117.994578,440.062954
14,678.279797,390.506710,26.715174,104.860961,323.797729
15,567.933067,482.443536,33.618790,93.522582,300.249012
16,450.480970,498.249602,43.113161,115.287528,375.204287
17,397.942282,470.694872,51.265421,101.417619,280.800440


## 웹 API

In [1]:
import requests # HTTP client

In [21]:
# API 요청
url = 'http://openapi.naver.com/search'
naver_search_api_test_key = 'c1b406b32dbbbbeee5f2a36ddc14067f'
params = {'key': naver_search_api_test_key,
         'target': 'news',
         'query': u'빅데이터',
         'display': 10}
res = requests.get(url, params=params)
print(res.status_code)
print(res.text)

200
<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0"><channel><title>Naver Open API - news ::'sample'</title><link>http://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Tue, 07 Jul 2015 13:04:24 +0900</lastBuildDate><total>3812</total><start>1</start><display>10</display><item><title>Eyelike: Joy Williams, Neil Young + Promise of the Real, Pete Rock</title><originallink>http://www.koreaherald.com/view.php?ud=20150701001246</originallink><link>http://openapi.naver.com/l?AAABWIuw6CMBRAv+Z2JPQBtEMHeSXGwVEcablIYoBaqsjfW0/OcHJeb/SHhqaEUoHM/iFrKFNoKlAxBDQ1SBEPCYdDveC+kSce2lIj0txwNpgIImYj63k+DJbGX4xk8jjqKQQH/ASsjW7YezslS/9Bn9h1JkFTwSWlOSsUVZLM2p2vnevEepOIe7vczSUHVla4fh9cAq9/+Ik2uq8AAAA=</link><description>... On one track a vocal &lt;b&gt;sample&lt;/b&gt; punctuates his agenda: “Longevity, baby.” (AP) 더 많은 기사 보러가기 → [코리아헤럴드 홈페이지] [트위터] [페이스북] 구독신청 1588... </description><pubDate>Fri, 03 Jul 2015 20:53:00 +0900</pubDate></item><item><title>Soybean m

## 데이터베이스 입출력

### sqlite3 활용

파이썬에는 sqlite3 DB가 기본적으로 탑재되어 있다.

https://docs.python.org/2/library/sqlite3.html

데이터베이스 생성

In [52]:
import sqlite3

query="""
create table test
(a varchar(20), b varchar(20), c real, d integer);
"""

# In-Memory DB
conn = sqlite3.connect(':memory:')
conn.execute(query)
conn.commit()

데이터베이스에 모의 데이터 몇 개 삽입

In [53]:
data = [(u'서울', u'경기', 1.25, 6),
        (u'대전', u'충청', 2.6, 3),
        (u'부산', u'경남', 1.7, 5)]
stmt = "insert into test values(?,?,?,?)"

conn.executemany(stmt, data)
conn.commit()

데이터베이스에 대해 질의 수행 후, 커서 가져와 읽기

In [57]:
# ... 결론은 이렇게 안 해도 된다는 것

pandas의 DataFrame으로 바로 읽어오기

<s>read_frame</s> 대신 read_sql

In [56]:
import pandas.io.sql as sql

sql.read_sql('select * from test', conn)

,a,b,c,d
0,서울,경기,1.25,6
1,대전,충청,2.60,3
2,부산,경남,1.70,5


### MongoDB 활용

MongoDB는 파이썬에 기본적으로 탑재되어 있지 않다. https://www.mongodb.org/ 에서 다운로드받아 설치할 수 있다.

In [8]:
import pymongo # MongoDB 드라이버
client = pymongo.MongoClient() # 로컬 서버, 기본 포트 27017